In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.regularizers as reg
import keras_tuner as kt
import numpy as np

import kaggle_functions as kaggle
import importlib
importlib.reload(kaggle);

In [3]:
train_dataset, test_dataset, test_labels = kaggle.load_train_as_dataset(resnet=True)
x_test_real = kaggle.load_test_set()

batch_size = 64
epoch_length = len(train_dataset) / batch_size
train_dataset_augmented = kaggle.augment_dataset(train_dataset, batch_size)

type_of_model = 'model5'

### ResNet

In [5]:
class ResNet(kt.HyperModel):
    def build(self, hyperparameters):
        dense_l2_reg = 0.01
        final_dropout = 0.7

        model = keras.applications.ResNet50V2(
            include_top=False, weights=None, 
            input_shape = (96, 96, 3), pooling='avg'
        )
        input = model.input
        output = model.output
        output = layers.Dropout(0.7)(final_dropout)
        output = layers.Dense(11, kernel_regularizer=keras.regularizers.l2(dense_l2_reg))(output)

        model = keras.Model(inputs=input, outputs=output)

        # Create model
        model.compile(
            optimizer=keras.optimizers.Nadam(0.0001),
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

        return model

ResNet().build(None).summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 102, 102, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 48, 48, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 50, 50, 64)   0           ['conv1_conv[0][0]']             
                                                                                              

In [ ]:
# Hyperparameter tuning

reload_tuner = True
tuner_filepath = 'hypertuner_2021-11-25'

tuner_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
]

tuner = kt.BayesianOptimization(ResNet(),
    objective='val_accuracy',
    max_trials=50,
    seed=1,
    directory=f'models/{type_of_model}',
    project_name=tuner_filepath,
    overwrite=(not reload_tuner))

tuner.search_space_summary()

if reload_tuner:
    tuner.reload()
else:
    tuner.search(
        train_dataset.batch(64).cache(), 
        validation_data=test_dataset.batch(64).cache(),
        epochs=14, callbacks=tuner_callbacks, verbose=1)

tuner.results_summary()

best_hyperparameters = tuner.get_best_hyperparameters()[0]
best_model = tuner.get_best_models()[0]
best_model.save(f'models/{type_of_model}/{tuner_filepath}/saved_model')

In [6]:
# Load existing model if wanted, otherwise create new
use_existing_model = False
model_name = 'resnet_1'
if use_existing_model:
    print("Loading existing model")
    model = keras.models.load_model(f'models/{type_of_model}/{model_name}')
else:
    print("Creating new model")
    model = ResNet().build(None)

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
]

# Fit model (can continue from loaded weights)
history = model.fit(
    train_dataset_augmented, 
    validation_data=test_dataset.batch(128).cache(),
    epochs=100, steps_per_epoch=epoch_length,
    callbacks=callbacks, verbose=1)

Creating new model
Epoch 1/100
148/148 [==============================] - 185s 1s/step - loss: 2.2880 - accuracy: 0.2038 - val_loss: 2.5543 - val_accuracy: 0.0627
Epoch 2/100
148/148 [==============================] - 156s 1s/step - loss: 2.2046 - accuracy: 0.2484 - val_loss: 2.9903 - val_accuracy: 0.1661
Epoch 3/100
148/148 [==============================] - 151s 1s/step - loss: 2.1375 - accuracy: 0.2738 - val_loss: 3.1174 - val_accuracy: 0.0988
Epoch 4/100
148/148 [==============================] - 93s 620ms/step - loss: 2.0317 - accuracy: 0.3107 - val_loss: 3.1942 - val_accuracy: 0.0921
Epoch 5/100
148/148 [==============================] - 42s 282ms/step - loss: 1.9839 - accuracy: 0.3234 - val_loss: 6.8861 - val_accuracy: 0.1585
Epoch 6/100
148/148 [==============================] - 46s 310ms/step - loss: 1.9299 - accuracy: 0.3456 - val_loss: 5.2527 - val_accuracy: 0.1501
Epoch 7/100
148/148 [==============================] - 139s 939ms/step - loss: 1.8869 - accuracy: 0.3640 - val_

KeyboardInterrupt: 

In [ ]:
# Save model
model.save(model_name)

kaggle.plot_model_history(history, ['accuracy', 'loss', 'val_accuracy', 'val_loss'])

### Evaluate model

In [ ]:
model_to_evaluate = model_name # Can be changed to evaluate older models
model = keras.models.load_model(f'models/model4/{model_to_evaluate}')

test_pred_raw = model.predict(test_dataset.batch(128))
test_pred = np.argmax(test_pred_raw, axis=1)

kaggle.print_accuracy(test_labels, test_pred)
kaggle.plot_confusion_matrix(test_labels, test_pred)

### Get labels for Kaggle

In [11]:
true_test_pred = np.argmax(model.predict(x_test_real), axis=1)

kaggle.save_test_pred(f'models/{type_of_model}/{model_name}_test_pred.csv', true_test_pred)